# Deploy

Utilities to deploy app

In [ ]:
# default_exp deploy

In [ ]:
# export
import json
import requests
from textwrap import dedent

def deploy(app="app.py", from_jupyter=True):
    """Deploy a streamlit app using ngrok"""
    if from_jupyter:
        get_ipython().system_raw('ngrok http 8501 &')
        resp = requests.get("http://localhost:4040/api/tunnels")
        tunnel = json.loads(resp.content)["tunnels"][0]
        local = tunnel["config"]["addr"]
        public = tunnel["public_url"]
    else:
        raise NotImplementedError(f"Deployment outside jupyter currently not supported")

    print(dedent(f"""\
        1. Create a new cell with: !nohup streamlit run {app}
        2. Run that cell
        3. Click on this link: {public}

      Note: this will link to local address {local}
    """))

In [ ]:
# exporti
import os
import pathlib
import re
from contextlib import contextmanager
from typing import Union

from fastai.learner import load_learner, Learner


@contextmanager
def set_posix():
    """To be able to load model in Windows"""
    posix_backup = pathlib.PosixPath
    try:
        if os.name == "nt":
            pathlib.PosixPath = pathlib.WindowsPath
        yield
    finally:
        pathlib.PosixPath = posix_backup


In [ ]:
# export
PathStr = Union[pathlib.Path, str]

def get_learner(model_path: PathStr) -> Learner:
    try:
        with set_posix():
            return load_learner(model_path)
    except AttributeError as e:
        m_missing_func = re.match(r"Can't get attribute '(.*?)'", str(e))
        if m_missing_func:
            raise AttributeError(
                f"Add in the app the implementation of '{m_missing_func.group(1)}'"
            )
        else:
            raise
